# Can We Predict Restaurent Sucees based on Location ?

## Introduction
Yelp academic data set provides wealth of information about businesses in 5 cities. This data set will be used to build a prdictive model to see how well a restaurent success is based on location. The dataset consist of five differenct types of data about a restaurent.
1. About the restaurent such as name, precise location, address, overall rating,and catagory.
2. The reviews of each restaurents provided by customers.
3. some information about the custormer, such as how many reviews, average starts given, friends network in yelp
4. check ins by customers
5. tips given by customers
***provide more information about dataset ***
*business.json file contains information about all the businesses in the dataset.

## Problem address by this project
How well a surronding commercial value of a location of a restaurent can predict the restaurent success.
1. How do we define a restaurent is successfull ?
Location:
The choice of location is expected to have significant impact on the success or failure of a restaurant. Success of a location depends not only on its physical site, but also its surrounding demographics. In other words, location is a complex construct that encompasses geographic, as
well as demographic and psychographic variables. Any changes in geographic or demographic/psychographic variables of a specific location could have significant impact on the attractiveness of a location. For example, Kahiki Restaurant located in Columbus, Ohio was ranked as one of the Top 100 restaurants in the US. It was visited by several Hollywood and national celebrities as a destination restaurant. Over next three decades, this particular location continued to become less attractive as the affluent of the neighborhood moved, leaving empty spaces, vacant houses, and ‘brown fields’. Eventually Kahiki was closed for good as the location became impossible to operate profitably. In this case, the geographic factors of the location did not change much, but the demographics changed; resulting restaurant failure.
In another situation, a restaurant in Ohio was forced to close its operations after operating nearly 20 years in the same location. In this case, the demographics actually improved over the 20 years making the location very attractive. However, as this small town grew, the city government decided to build embankments on both sides of the road to improve public safety. These embankments totally obscured the restaurant from customers’ view. As a result, customers were forced to go to the next traffic light and take a turn to visit this restaurant; compromising the convenience factor which is so important for the industry. This restaurant was closed eventually due to reduced patronage as a result of the lack of visibil

In [2]:
#load libraries
import json
import csv
import pandas as pd
from geopy.distance import vincenty

From the yelp dataset load business data to a dataframe

In [17]:
business_file ="/Users/chathura/Documents/yelp_dataset_challenge_academic_dataset/LV_2biz.json"
bf = open(business_file,'r')
#Following list comprehension 
#output json from line for each line in file if the json object's categores has "Restaurents and state has 2 letters
#biz = [json.loads(line) for line in bf if 'Restaurants' in json.loads(line)['categories'] and (len(json.loads(line)['state'])==3)] 
#print (json.loads(bf.readline()))
#restaurents = [json.loads(line) for line in bf if 'Restaurants' in json.loads(line)['categories']]
#biz = [json.loads(line) for line in bf if ('Restaurants' in json.loads(line)['categories']) or ('Department Stores' in json.loads(line)['categories']) or ('Grocery' in json.loads(line)['categories']) or ('Shopping' in json.loads(line)['categories'])]
#res = [json.loads(line) for line in bf if ('Restaurants' in json.loads(line)['categories'])]
biz = [json.loads(line) for line in bf]
bf.close()
biz_df = pd.DataFrame(biz)


Zip code from the address is extracted for each business and added to the data frame

In [18]:
zip_code = [biz_df['full_address'][i].split(" ")[-1] for i in range(biz_df.shape[0])]
biz_df['zip_code'] = zip_code

In [19]:
street=[]
lot=[]
for i in range(biz_df.shape[0]):
    try:
        street.append(biz_df['attributes'][i]['Parking']['street'])
        #print (biz_df['attributes'][i]['Parking']['street'])
    except:
        street.append(False)
        
for i in range(biz_df.shape[0]):
    try:
        lot.append(biz_df['attributes'][i]['Parking']['lot'])
    except:
        lot.append(False)


In [20]:
biz_df['street'] = street
biz_df['lot'] = lot

In [21]:
biz_df.shape[0]

4132

In [ ]:
wh = open('./file_lv.csv','w')
for i in range(biz_df.shape[0]):
    if("Restaurants" in biz_df['categories'][i]):
        print (i)
        inf_biz = influence_biz(biz_df['business_id'][i])
        score = biz_df[biz_df['business_id']==biz_df['business_id'][i]]['review_count'].tolist()[0]*biz_df[biz_df['business_id']==biz_df['business_id'][i]]['stars'].tolist()[0]
        #print(biz_df['street'][i],biz_df['lot'][i],influence_biz(biz_df['business_id'][i]),]
        seq = [str(biz_df['street'][i]),str(biz_df['lot'][i]),str(inf_biz[0]),str(inf_biz[1]),str(inf_biz[2]),str(inf_biz[3]),str(inf_biz[4]),str(score),"\n"]
        wh.write(",".join(seq))
        #res_list.append(i)
wh.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25

In [36]:
biz_df[biz_df['business_id']==biz]['stars']

Series([], Name: stars, dtype: float64)

In [84]:
influence_biz("2ZnCITVa0abGce4gZ6RhIw")

[0, 190.5, 1084.5, 0, 192.0]

In [9]:
def influence_biz(biz_id):
    # This function retures biznesses that are 2 minit walking disctance from given bizness.
    infbiz=[]
    row = biz_df[biz_df['business_id']==biz_id]
    zip = list(row['zip_code'])
    lat1 = list(row['latitude'])
    lon1 = list(row['longitude'])
    biz1 = (lat1[0],lon1[0])
    nearby = biz_df[biz_df['zip_code']==zip[0]]
    for biz in range(nearby.shape[0]):
        near_biz_id = list(nearby[0:]['business_id'])[biz]
        if(near_biz_id==biz_id):#this will avoid self counting as a near business
            continue
        lat2 = list(biz_df[biz_df['business_id']==near_biz_id]['latitude'])
        lon2 = list(biz_df[biz_df['business_id']==near_biz_id]['longitude'])
        biz2 = (lat2[0],lon2[0])
        if(vincenty(biz1, biz2).miles < 0.2):
            infbiz.append(near_biz_id)
    
    dep_stores = 0
    restaurents=0
    shopping = 0
    hotels = 0
    arts=0
    
    for biz in infbiz:
        row = biz_df[biz_df['business_id']==biz]
        score = row['stars'].tolist()[0]*row['review_count'].tolist()[0]
        if("Department Stores" in row['categories'].tolist()[0]):
            dep_stores=dep_stores+score
            continue
        if("Restaurants" in row['categories'].tolist()[0]):
            restaurents=restaurents+score
            continue
        if("Shopping" in row['categories'].tolist()[0]):
            shopping=shopping+score
            continue
        if("Hotels" in row['categories'].tolist()[0]):
            hotels=hotels+score
            continue
        if("Arts & Entertainment" in row['categories'].tolist()[0]):
            arts=arts+score
            continue
    inf_biz_list=[dep_stores,restaurents,shopping,hotels,arts]
    return inf_biz_list

In [74]:
row = biz_df[biz_df['business_id']=="werWnE_3k8GYVaMuUcWw4g"]
row['city']

7    Phoenix
Name: city, dtype: object

is there a high scoring business near by ?
school near by
avenue,blve,street,drive
Number of potentioal customers in the vicinity

In [64]:
biz_df[biz_df['business_id']=="u0NbikWwP7TVkMkCily-4g"][]

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,zip_code,street,lot
1980,"{'Parking': {'valet': False, 'street': False, ...",u0NbikWwP7TVkMkCily-4g,"[Museums, Arts & Entertainment, Art Galleries,...",Phoenix,"1625 N Central Ave\nPhoenix, AZ 85004","{'Friday': {'open': '10:00', 'close': '17:00'}...",33.467302,-112.073412,Phoenix Art Museum,[],True,241,4.5,AZ,business,85004,False,True


In [16]:
wh = open('./restaurent_locations_LV.csv','w')
for i in range(biz_df.shape[0]):
    print(i)
    if("Restaurants" in biz_df['categories'][i] and "Las Vegas" in biz_df['city'][i] ):
        score = biz_df[biz_df['business_id']==biz_df['business_id'][i]]['review_count'].tolist()[0]*biz_df[biz_df['business_id']==biz_df['business_id'][i]]['stars'].tolist()[0]
        seq = [str(biz_df['business_id'][i]),str(biz_df[biz_df['business_id']==biz_df['business_id'][i]]['latitude'].tolist()[0]),str(biz_df[biz_df['business_id']==biz_df['business_id'][i]]['longitude'].tolist()[0]),str(score),"\n"]
        wh.write(",".join(seq))        
wh.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [1]:
review_file  = "/Users/chathura/Documents/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json"
rf = open(review_file,'r')
review = [json.loads(line) for line in rf]
rf.close()
review_df = pandas.DataFrame(review)
review_colnames = review_df.columns.values.tolist()
print (review_colnames)
    

NameError: name 'json' is not defined

In [25]:
import json
from geopy.distance import vincenty

In [16]:
reduced_catagories = ['Restaurants','Food','Beauty\&Spas','Bars','Nightlife','Arts\&Entertainment','EventPlanning&Services'
                      ,'Fitness&Instruction',
                      'ActiveLife','LocalServices','ProfessionalServices','PublicServices&Government','FinancialServices',
                      'HomeServices',
                      'AutoRepair','Automotive','Pets','Health&Medical','Shopping','SpecialtySchools','Hotels&Travel',
                      'ReligiousOrganizations']
len(reduced_catagories)
#what location based factors affect a restaurent rating ?
near_by_influence = []
#define : an geographic area which people come and go freuently do do shopping,see a movie,visit a patient
# near a department store. # near a auto repair, neas school,near motels and hotels 
#shopping,other_Restaurants_same_type,other_restaurents_different_type,fastFood_restaruents,near_school,motels,small_hotels
#auto_repair # easy_parking 

In [321]:




newport_ri = (40.4140637, -79.9037778)
cleveland_oh = (40.411697799999999, -79.912408200000002)
print(vincenty(newport_ri, cleveland_oh).miles)



0.48355803583654156


In [21]:
for fname in ( '/Users/chathura/Documents/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json',):
    with open(fname, 'r+') as f:
        lineno = 0
        # this reads in one line at a time from stdin
        for line in f:
            lineno += 1
            #print ('{:>6} {}'.format(lineno, line))
            j = json.loads(line)
            if 'Restaurants' in j['categories']:
                print (j['full_address'])
                print ('{:>6} {} {}'.format(lineno,j['business_id'],j['stars']),j['latitude'],j['longitude'])
                
            
            #print j['review_id']
            #print j['business_id']
            #print j['stars']
            #print j['text']
            #print j['votes']
            #print "---"
            
            if lineno == 1000:
                break


414 Hawkins Ave
Braddock, PA 15104
     5 mVHrayjG3uZ_RLHkLj-AMg 4.5 40.408735 -79.8663507
141 Hawthorne St
Greentree
Carnegie, PA 15106
     6 KayYbHCt-RkbGcPdGOThNg 4.0 40.415517 -80.067534
2100 Washington Pike
Carnegie, PA 15106
    10 wJr6kSA5dchdgOdwH6dZ2w 3.5 40.3877323 -80.0928745
1201 Washington Ave
Carnegie, PA 15106
    12 fNGIbpazjTRdXgwRY_NIXA 4.0 40.3964688 -80.0849416
1073 Washington Ave
Carnegie, PA 15106
    13 b9WZJp5L1RZr4F1nxclOoQ 4.5 40.3967441 -80.0847998
202 3rd Ave
Carnegie
Carnegie, PA 15106
    14 zaXDakTd3RXyOa7sMrUE1g 4.0 40.404638 -80.089985
215 E Main St
Carnegie
Carnegie, PA 15106
    16 WETE_LykpcnrC1sFcQ5EGg 3.5 40.40867 -80.085279
Raceway Plz
Carnegie, PA 15106
    19 rv7CY8G_XibTx82YhuqQRw 3.5 40.3868915 -80.0937037
214 E Main St
Carnegie
Carnegie, PA 15106
    21 SQ0j7bgSTazkVQlF5AnqyQ 2.5 40.408343 -80.084861
2180 Greentree Rd
Pittsburgh, PA 15220
    22 wqu7ILomIOPSduRwoWp4AQ 4.0 40.391255 -80.073426
200 E Main St
Carnegie
Carnegie, PA 15106
    24 

## Introduction
How important location for the sucess of a business.How big of a factor it plays in terms of accesiblity to the business, parking availablitiy, traffic. crowd management in the the visinity.

Yelp has wealth of information about businesses in cities. So from the review we should be able to pickup some information about locations to look for a parking spots for a business in a busy streets. The algorithm should use  reviews, tips, and location of the business to try to find a parking spot. Natural Language processing techniques should be used to find information from reviews. Location mining can be used to find busy streets. And the parking spot should be less than 1/4 of a mile.The average person's stride length is approximately 2.5 feet long. That means it takes just over 2,000 steps to walk one mile. So a person has to walk about 500 steps from parking to business.

In [3]:
import json
# a simple filter that prepends line numbers
# import sys EDIT: unused, pointed out in comments here and on HN
for fname in ( '/Users/chathura/Documents/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json',):
    with open(fname, 'r+') as f:
        lineno = 0
        # this reads in one line at a time from stdin
        for line in f:
            lineno += 1
            print ('{:>6} {}'.format(lineno, line))
            #j = json.loads(line)
            ##print j['user_id']
            #print j['review_id']
            #print j['business_id']
            #print j['stars']
            #print j['text']
            #print j['votes']
            #print "---"
            
            if lineno == 10:
                break


     1 {"business_id": "vcNAWiLM4dR7D2nwwJ7nCA", "full_address": "4840 E Indian School Rd\nSte 101\nPhoenix, AZ 85018", "hours": {"Tuesday": {"close": "17:00", "open": "08:00"}, "Friday": {"close": "17:00", "open": "08:00"}, "Monday": {"close": "17:00", "open": "08:00"}, "Wednesday": {"close": "17:00", "open": "08:00"}, "Thursday": {"close": "17:00", "open": "08:00"}}, "open": true, "categories": ["Doctors", "Health & Medical"], "city": "Phoenix", "review_count": 9, "name": "Eric Goldberg, MD", "neighborhoods": [], "longitude": -111.98375799999999, "state": "AZ", "stars": 3.5, "latitude": 33.499313000000001, "attributes": {"By Appointment Only": true}, "type": "business"}

     2 {"business_id": "UsFtqoBl7naz8AVUBZMjQQ", "full_address": "202 McClure St\nDravosburg, PA 15034", "hours": {}, "open": true, "categories": ["Nightlife"], "city": "Dravosburg", "review_count": 4, "name": "Clancy's Pub", "neighborhoods": [], "longitude": -79.886930000000007, "state": "PA", "stars": 3.5, "latitud

In [2]:
import json
# a simple filter that prepends line numbers
# import sys EDIT: unused, pointed out in comments here and on HN
for fname in ( '/Users/chathura/Documents/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json',):
    with open(fname, 'r+') as f:
        lineno = 0
        # this reads in one line at a time from stdin
        for line in f:
            lineno += 1
            #print '{:>6} {}'.format(lineno, line)
            j = json.loads(line)
            print (j['user_id'])
            print (j['review_id'])
            print (j['business_id'])
            print (j['stars'])
            print (j['text'])
            print (j['votes'])
            print ("---")
            
            if lineno == 100 :
                break

Xqd0DzHaiyRqVH3WRG7hzg
15SdjuK7DmYqUAj6rjGowg
vcNAWiLM4dR7D2nwwJ7nCA
5
dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
{'funny': 0, 'useful': 2, 'cool': 1}
---
H1kH6QZV7Le4zqTRNxoZow
RF6UnRTtG7tWMcrO2GEoAg
vcNAWiLM4dR7D2nwwJ7nCA
2
Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an a

     1 {"business_id": "vcNAWiLM4dR7D2nwwJ7nCA", "full_address": "4840 E Indian School Rd\nSte 101\nPhoenix, AZ 85018", "hours": {"Tuesday": {"close": "17:00", "open": "08:00"}, "Friday": {"close": "17:00", "open": "08:00"}, "Monday": {"close": "17:00", "open": "08:00"}, "Wednesday": {"close": "17:00", "open": "08:00"}, "Thursday": {"close": "17:00", "open": "08:00"}}, "open": true, "categories": ["Doctors", "Health & Medical"], "city": "Phoenix", "review_count": 9, "name": "Eric Goldberg, MD", "neighborhoods": [], "longitude": -111.98375799999999, "state": "AZ", "stars": 3.5, "latitude": 33.499313000000001, "attributes": {"By Appointment Only": true}, "type": "business"}

     2 {"business_id": "UsFtqoBl7naz8AVUBZMjQQ", "full_address": "202 McClure St\nDravosburg, PA 15034", "hours": {}, "open": true, "categories": ["Nightlife"], "city": "Dravosburg", "review_count": 4, "name": "Clancy's Pub", "neighborhoods": [], "longitude": -79.886930000000007, "state": "PA", "stars": 3.5, "latitud

Tips mining for businesses: location related tips for rastarutens in busy suberbs

Tip :
{"user_id": "KoplwVOqt46ON4MWnTb_4g", "text": "There is a lot of parking in the back of you can't find a space in the front.", "business_id": "IqbY-ztC3GC_A_KoKhYKuA", "likes": 0, "date": "2014-12-02", "type": "tip"}


Review :
{"votes": {"funny": 5, "useful": 7, "cool": 3}, "user_id": "m7vtKWpZ9wdEQ95wJxrMrg", "review_id": "czWOtAkRhO4P90rOKAvRbA", "stars": 5, "date": "2014-11-24", "text": "Good, inexpensive Thai cuisine, well presented and no corners cut even as they keep their prices down. The service is friendly and the place is cozy and conveniently located. While parking in front of this commercial strip facing St. Joe's hospital is very limited, there is ample parking behind - as is known by those who frequent the popular 5th Avenue Cafe which happens to be their neighbor.", "type": "review", "business_id": "IqbY-ztC3GC_A_KoKhYKuA"}

Issue: need to identify a busy street based on businesses around it. may be to find the busiest time of the day and find a tip according to the situation.

ImportError: No module named 'osgeo'